In [4]:
from sklearn.linear_model import LogisticRegression
import scipy.io
import numpy as np
import h5py
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import torch.nn as nn
from scipy.stats.stats import pearsonr
import torch
from matplotlib.pyplot import figure


In [5]:
class DataManager():
    
    def __init__(self, params, num_bands):
        
        self.rl_data = None
        self.dataset_type = params['dataset_type']
        self.data_file_path = params['data_file_path']
        self.num_bands = num_bands
        
        #load the data
        assert self.dataset_type in ('IndianPines', 'Botswana', 'SalientObjects'), f'{self.dataset_type} is not valid' 
        #separating out in case any of the data requires unique pre-processig
        if self.dataset_type == 'IndianPines':
            self.load_indian_pine_data()
        elif self.dataset_type == 'Botswana':
            self.load_botswana_data()
        elif self.dataset_type == 'SalientObjects':
            self.load_salient_objects_data()
            
        #self.x_train = None
        #self.y_train = None
        #self.x_test = None
        #self.y_test = None

    def load_indian_pine_data(self):        

        hyper_path = '../data/indian_pines/hyperspectral_imagery/indian_pines_corrected.npy'
        hyper = np.load(hyper_path)
        print(hyper.shape)

        # randomly sample for x% of the pixels
        indices = np.random.randint(0, hyper.shape[0], int(hyper.shape[0]*0.01))
        self.rl_data = hyper[indices, :]
        print(self.rl_data.shape)


    def load_salient_objects_data(self):        

        hyper_path = '../data/salient_objects/hyperspectral_imagery/0001.npy'
        hyper = np.load(hyper_path)
        print(hyper.shape)

        # randomly sample for x% of the pixels
        indices = np.random.randint(0, hyper.shape[0], int(hyper.shape[0]*0.01))
        self.rl_data = hyper[indices, :]
        print(self.rl_data.shape)

        
    def load_botswana_data(self):
        
        self.rl_data = scipy.io.loadmat(self.data_file_path)

    #def load_salient_objects(self)
        

In [6]:
class ReplayBuffer():
    
    def __init__(self, size=10000):
        self.size = size
        self.paths = []
        
    def add_trajectories(self, paths):
        self.paths.extend(paths)
        self.paths = self.paths[-self.size:]
        
    def sample_buffer_random(self, num_trajectories):
        
        rand_idx = np.random.permutation(len(self.paths))[:num_trajectories]
        return [self.paths[i] for i in rand_idx]
        #return self.paths[rand_idx]
    
        

In [7]:
class Agent():
    
    def __init__(self, params):
        
        self.agent_params = params['agent']
        self.num_bands = self.agent_params['num_bands']
        self.n_iter = self.agent_params['n_iter']
        self.trajectory_sample_size = self.agent_params['trajectory_sample_size']
        self.batch_size = self.agent_params['batch_size']
        self.num_critic_updates = self.agent_params['num_critic_updates']
        
        valid_rewards = ['correlation', 'mutual_info']
        assert self.agent_params['reward_type'] in valid_rewards, 'rewards must be one of ' + valid_rewards.join(',') 
        
        if self.agent_params['reward_type'] == 'correlation':
            self.reward_func = self.calculate_correlations
        elif self.agent_params['reward_type'] == 'mutual_info':
            self.reward_func = self.calculate_mutual_infos
        
        self.data_params = params['data']
        self.DataManager = DataManager(self.data_params, self.num_bands)
        self.band_selection_num = self.data_params['band_selection_num']

        self.critic_params = params['critic']
        self.critic = QCritic(self.critic_params, self.num_bands)
        
        
        self.policy_params = params['policy']
        self.policy = ArgMaxPolicy(self.policy_params, self.critic)
        
        self.replay_buffer = ReplayBuffer()
        
        self.cache = {}
        
        self.logging_df = pd.DataFrame()
        
    
    def generateTrajectories(self):
        
        #we expect paths to be a list of trajectories
        #a trajectory is a list of Path objects
        paths = []
        for i in range(self.trajectory_sample_size):
            
            path = self.sampleTrajectory()
#             print(f'Iter {i}')
#             print([p['re'] for p in path])
            
            paths.append(path)
    
        return paths
    
    def sampleTrajectory(self, iter_num = 1):
            
        #select 30 actions
        state = np.zeros(self.num_bands)
        state_next = state.copy()
        
        #paths will be a list of dictionaries
        path = []
        for i in range(self.band_selection_num):
            
            action, action_type = self.policy.get_action(state)
            state_next[action] += 1

            reward, correlation_current_state, correlation_next_state = self.calculate_reward(state, state_next)

            terminal = 1 if i == self.band_selection_num - 1 else 0
            path.append(self.Path(state.copy(), action, state_next.copy(), reward, terminal))
            
            state = state_next.copy()
        
            if iter_num % 100 == 0:
                print("Iter : ", iter_num)
                q_values = self.critic.get_action(state)
                
                sampled_paths = self.replay_buffer.sample_buffer_random(1)
                
                flat_sampled_path = [path for trajectory in sampled_paths for path in trajectory]
                obs = np.array([path['ob'] for path in flat_sampled_path])
                acs = np.array([path['ac'] for path in flat_sampled_path])
                obs_next = np.array([path['ob_next'] for path in flat_sampled_path])
                res = np.array([path['re'] for path in flat_sampled_path])
                terminals = np.array([path['terminal'] for path in flat_sampled_path])
                
                loss_value = self.critic.update(obs, acs, obs_next, res, terminals)
                
                row = {
                    "iter_num": iter_num,
                    "Selected Band": i,
                    "Action Type": action_type,
                    "Mean": torch.mean(q_values).detach().numpy(),
                    "Min": torch.min(q_values).detach().numpy(),
                    "Max": torch.max(q_values).detach().numpy(),
                    "Correlation Current State" : correlation_current_state,
                    "Correlation Next State" : correlation_next_state,
                    "Reward" : reward,
                    "Loss" : loss_value
                }
                
                self.logging_df = self.logging_df.append(row, ignore_index=True)
                
                
#                 print(self.logging_df)
        
        #path returns state, action, state_next, reward, terminal
        return path
                   
        
    def runAgent(self):
        
        for iter_num in range(self.n_iter):
            
            print('Iteration ', iter_num, ':')
            
            paths = self.generateTrajectories()
            self.replay_buffer.add_trajectories(paths)
            
            for _ in range(self.num_critic_updates):
                sampled_paths = self.replay_buffer.sample_buffer_random(self.agent_params['batch_size'])
                
                flat_sampled_path = [path for trajectory in sampled_paths for path in trajectory]
                obs = np.array([path['ob'] for path in flat_sampled_path])
                acs = np.array([path['ac'] for path in flat_sampled_path])
                obs_next = np.array([path['ob_next'] for path in flat_sampled_path])
                res = np.array([path['re'] for path in flat_sampled_path])
                terminals = np.array([path['terminal'] for path in flat_sampled_path])
                
                critic_loss = self.critic.update(obs, acs, obs_next, res, terminals)
                
            self.critic.update_target_network()
            
            #sample a single trajectory
            print('------------------------------------EVAL Results------------------------------')
            eval_path = self.sampleTrajectory(iter_num)
#             print(self.cache)
            #print(eval_path[-1])
            #print('Selected_Bands: ', np.argwhere(eval_path[-1]['ob_next']>0))
            print('Num_Selected_Bands: ', np.argwhere(eval_path[-1]['ob_next']>0).shape[0])
            print('Eval_Return: ', np.sum(eval_path[-1]['re']))
            print('Critic_Loss: ', critic_loss)
                

    def calculate_reward(self, state, state_next):
        #for future, save down the previous state so that we can avoid a calc
        
#         print(list(np.argwhere(np.array(state) != 0)), list(np.argwhere(np.array(state_next) != 0)))
        if list(np.argwhere(np.array(state) != 0)) == list(np.argwhere(np.array(state_next) != 0)):
#             print("same action selected")
            return -1, "Indef", "Indef"
        else:
        
            a = self.reward_func(state)
            b = self.reward_func(state_next)
            return a-b, a, b
    
    
    def calculate_correlations(self, state):
        
#         if repr(state) in self.cache:
#             return self.cache[repr(state)]
        
        #deal with the first state
        ##### THIS LOGIC SEEMS WRONG - REGARDLESS OF THE FIRST PICK, YOU HAVE A REWARD OF 0#####
        if np.sum(state) <= 1:
            return 0
        
        selected_bands = []
        non_zero_bands = np.argwhere(np.array(state) != 0)
        for band in non_zero_bands:
#             print(band[0])
            selected_bands.extend([band[0]]*int(state[band[0]]))
        #print(selected_bands)
        #selected_bands = np.squeeze(np.argwhere(np.array(state)==1))
        corr_sum = 0
        for idx_i, i in enumerate(selected_bands):
            for idx_j, j in enumerate(selected_bands):
                if idx_i != idx_j:
                    
                    if repr((i,j)) in self.cache:
                        result = self.cache[repr((i,j))]
                    else:
                        result = abs(pearsonr(self.DataManager.rl_data[:, i], self.DataManager.rl_data[:, j])[0])
                        self.cache[repr((i,j))] = result
                    
                    corr_sum += result
                    
#                     corr_sum += abs(pearsonr(self.DataManager.rl_data[:, i], self.DataManager.rl_data[:, j])[0])
        
#         self.cache[repr(state)] = corr_sum/(len(selected_bands)**2)
        
#         return self.cache[repr(state)]
        return corr_sum/(len(selected_bands)**2)

    def calculate_mutual_infos(self, state):
    
        selected_bands = []
        non_zero_bands = np.argwhere(np.array(state) != 0)
        for band in non_zero_bands:
            selected_bands.extend([band[0]]*int(state[band[0]]))
    
        normalized_mutual_info_score_sum = 0
        for i in selected_bands:
            for j in selected_bands:

                normalized_mutual_info_score_sum += normalized_mutual_info_score(self.DataManager.rl_data[:, i],
                                                                                 self.DataManager.rl_data[:, j])

        return normalized_mutual_info_score_sum/(len(selected_bands)**2)

            
    def Path(self, ob, ac, ob_next, re, terminal):
        return {'ob':ob,
                'ac':ac,
                'ob_next':ob_next,
                're':re,
                'terminal':terminal
                }

In [8]:
class ArgMaxPolicy():
    
    def __init__(self, params, critic):
        self.epsilon = params['epsilon']
        self.epsilon_decay = params['epsilon_decay']
        self.critic = critic
        
    def get_action(self, obs):
        
        q_value_estimates = self.critic.get_action(obs)
        #print(obs)
#         print('Predicted Q-Values:', q_value_estimates)
        
        rand = np.random.rand()
        if rand < self.epsilon:
            #select a random action
#             print('Selected Random')
            unselected_bands = np.squeeze(np.argwhere(obs == 0))
            selected_idx = np.random.choice(unselected_bands)
            action_type = "Random Action"

        else:
#             print('Selected Max')
            selected_idx = torch.argmax(q_value_estimates).item()
            action_type = "Max Action"

            
        self.decay_epsilon()
        return selected_idx, action_type
                
    def decay_epsilon(self):
        self.epsilon *= self.epsilon_decay 

In [9]:
class QCritic():
    
    def __init__(self, params, num_bands):
        
        self.num_bands = num_bands

        
        self.critic = self.create_network()
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(),lr=0.005)

        self.critic_target = self.create_network()
        
        self.gamma = params['gamma']
        
        self.loss = nn.SmoothL1Loss()
    
    def create_network(self):
        
        q_net  = nn.Sequential(
        nn.Linear(self.num_bands, self.num_bands*2),
        nn.ReLU(),
        nn.Linear(self.num_bands*2, self.num_bands*2),
        nn.ReLU(),
        nn.Linear(self.num_bands*2, self.num_bands)
        )
        
        return q_net
    
        
    def forward(self, obs):
        # will take in one hot encoded states and output a list of qu values
        
        q_values = self.critic(obs)
        
        return q_values
    
    def get_action(self, obs):
        
        if isinstance(obs, np.ndarray):
            obs = from_numpy(obs)
            
        return self.critic(obs)
    
    def update(self, obs, ac_n, next_obs, reward_n, terminals):
        
        obs = self.check_tensor(obs) #comes in as shape 
        ac_n = self.check_tensor(ac_n)
        next_obs = self.check_tensor(next_obs)
        reward_n = self.check_tensor(reward_n)
        terminals = self.check_tensor(terminals)
        
        full_q_values = self.critic(obs)
        q_actions = full_q_values.argmax(dim=1)
        q_values = torch.gather(full_q_values, 1, q_actions.unsqueeze(1)).squeeze(1)
        
        
        #print('Obs ', obs.shape)
        #print('Full Q ', full_q_values.shape)
        #print('Q Actions ', q_actions.shape)
        #print('Q Val ', q_values.shape)
        
        full_q_next_target = self.critic_target(next_obs)
        q_actions_next = self.critic(next_obs).argmax(dim=1)
        #q_values_next = full_q_next.max(dim=1)
        #print('q_values_next', q_values_next)
        q_values_next = torch.gather(full_q_next_target, 1, q_actions_next.unsqueeze(1)).squeeze(1)
        
        #print('reward', type(reward_n.shape))
        #print('q_values_next', type(q_values_next))
        #print('terminals', type(terminals))
        #print('gamma', type(self.gamma))
        target = reward_n + self.gamma*q_values_next*(1-terminals)
        target = target.detach()
        
        #print(f'Target Dim: {target.shape}')
        #print(f'Q_Values Dim: {q_values.shape}')
        loss = self.loss(q_values, target)
        
        self.critic_optimizer.zero_grad()
        loss.backward()
        self.critic_optimizer.step()
        
        return loss.item()
    
    def check_tensor(self, ar):
        
        if isinstance(ar, np.ndarray):
            ar = from_numpy(ar)
            
        return ar
    
    def update_target_network(self):
        for target_param, param in zip(
            self.critic_target.parameters(), self.critic.parameters()):
            target_param.data.copy_(param.data)
        
        

In [10]:
#utility functions
## taken from Prof. Sergey Levine's CS285 HW

device = 'cpu'

def from_numpy(*args, **kwargs):
    return torch.from_numpy(*args, **kwargs).float().to(device)

def to_numpy(tensor):
    return tensor.to('cpu').detach().numpy()

In [11]:
params = {'agent':{
            'n_iter':2000,
            'trajectory_sample_size': 10,
            'batch_size':5,
            'num_critic_updates':5,
            'num_bands':200,
            'reward_type':'mutual_info'
            },
          'data':{
            'band_selection_num':30,
            'dataset_type':'SalienObjects',
            'data_file_path':r'../data/indian_pines/hyperspectral_imagery/indian_pines_corrected.npy',
            },
          'critic':{
            'gamma':0.99
            },
          'policy':{
            'epsilon':1,
            'epsilon_decay':0.999
            }
         }

In [12]:
agent = Agent(params)
agent.runAgent()

AttributeError: 'Agent' object has no attribute 'agent_param'

In [ ]:
agent.logging_df

In [ ]:
for i in range(6):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    plot_df = agent.logging_df[agent.logging_df["Selected Band"] == i]
    ax1.plot(plot_df["iter_num"], plot_df["Mean"], color='red', label="Mean")
    ax1.plot(plot_df["iter_num"], plot_df["Max"], color='blue', label="Max")
    ax1.plot(plot_df["iter_num"], plot_df["Min"], color='green', label="Min")
    ax1.axhline(plot_df['Reward'].mean(), color='red')
    ax1.set_title(f'Band Selection {i}')
    ax1.legend()
    ax2.plot(plot_df["iter_num"], plot_df["Loss"], color='red')
    ax2.set_title('Loss Function')
    plt.show()

In [ ]:
agent.logging_df

In [ ]:
band_1_logging_df = agent.logging_df[agent.logging_df["Selected Band"] == 1.0]

In [ ]:
band_1_logging_df

In [ ]:
plt.figure(num = 3, figsize=(8, 5))
plt.plot(band_1_logging_df["iter_num"], band_1_logging_df["Mean"], color='red', label="Mean")
plt.plot(band_1_logging_df["iter_num"], band_1_logging_df["Max"], color='blue', label="Max")
plt.plot(band_1_logging_df["iter_num"], band_1_logging_df["Min"], color='green', label="Min")

plt.axhline(band_1_logging_df['Reward'].mean( ))

plt.xlabel("iter num")
plt.ylabel("Q values")
plt.legend(loc="upper left")

plt.show()

In [ ]:
plt.figure(num = 3, figsize=(8, 5))
plt.plot(band_1_logging_df["iter_num"], band_1_logging_df["Loss"], color='red')
plt.show()


In [ ]:
path = '/Users/romitbarua/Documents/Berkeley/Fall 2022/CS285-Deep Reinforcement Learning/HyperSpectralRL/data/data_indian_pines_drl.mat'
data = scipy.io.loadmat(path)['x']
band_1 = data[:, 173]
band_2 = data[:, 2]

In [ ]:
num = np.sum((band_1 - np.mean(band_1))*(band_2 - np.mean(band_2)))
den = np.sqrt(np.sum((band_1 - np.mean(band_1))**2)*np.sum((band_2 - np.mean(band_2))**2))
num/den

In [ ]:
band_1

In [ ]:
band_2

In [ ]:
corr = []
band_1 = data[:, 40]
for i in range(2, 200):

    band_2 = data[:, i]
    num = np.sum((band_1 - np.mean(band_1))*(band_2 - np.mean(band_2)))
    den = np.sqrt(np.sum((band_1 - np.mean(band_1))**2)*np.sum((band_2 - np.mean(band_2))**2))
    corr.append(num/den)
    
    
sns.histplot(corr)

In [ ]:
path = r'/Users/pratikaher/FALL22/HyperSpectralRL/ForPratik/data_indian_pines_drl.mat'
num_iter = 1000
num_bands = 20
bands_to_select = 5
data = scipy.io.loadmat(path)['x'][:, :num_bands]

all_band_selection = []
all_corr = []
all_reward = []

for num_iter in range(num_iter):
    selected_bands = list(np.random.choice(np.arange(0, num_bands, 1), 1))
    corr = [0]
    rewards = [0]
    for i in range(bands_to_select):

        selected_bands.extend(list(np.random.choice(np.arange(0, num_bands, 1), 1)))
        corr_sum = 0
        for idx_i, i in enumerate(selected_bands):
            for idx_j, j in enumerate(selected_bands):
                if idx_i != idx_j:
                    corr_sum += abs(pearsonr(data[:, i], data[:, j])[0])
        corr.append(corr_sum/(len(selected_bands)**2))
        rewards.append(corr[-2] - corr[-1])
    
    if num_iter % 100 == 0:
        print(num_iter)
        
    all_band_selection.append(selected_bands)
    all_corr.append(corr)
    all_reward.append(rewards)
    
all_band_selection = np.array(all_band_selection)
all_corr = np.array(all_corr)
all_reward = np.array(all_reward)

In [ ]:
num_iter = 1000
cum_reward = np.zeros((num_iter, bands_to_select))
for i in range(bands_to_select):
    cum_reward[:, i] = np.sum(all_reward[:, i+1:], axis=1)
    
# reasonbleness check to make sure this works
#print(all_reward[0, :])
#print(sum(all_reward[0, :]))
#print(cum_reward[0])

In [ ]:
for i in range(bands_to_select):
    sns.histplot(cum_reward[:,i])
    plt.title(f'Distribution of Q-Values at band selection {i+1}')
    plt.show()